# Oregon soil data compilation 
This notebook cleans and aggregates SSURGO data for use in Landmapper. The result is a spatial layer containing the soil attributes reported in the application. 

In [2]:
import pandas as pd
import geopandas as gpd
#import arcpy

In [3]:
SOILS = "G:/projects/LandMapper_2020/Data/Source/Soils/2023/OR/gSSURGO_OR.gdb"

In [8]:
# read in relevant tables
# Mapunit Aggregated Attributes
muaggatt = gpd.read_file(SOILS, driver='fileGDB', layer='muaggatt')
# Component
component = gpd.read_file(SOILS, driver='fileGDB', layer='component')
# Component Forest Productivity
coforprod = gpd.read_file(SOILS, driver='fileGDB', layer='coforprod')
# Component Restrictions
corestrictions = gpd.read_file(SOILS, driver='fileGDB', layer='corestrictions')

The muaggatt table has soil attribute aggregated to the map unit level.  This is convenient, as values from this table can be reported directly. 
We will pull name, drainage class, erosion hazard, and slope.

In [71]:
# let's start with ethe muaggatt table
muaggatt_sub = muaggatt[['mukey', 'muname', 'drclassdcd', 'forpehrtdcp']]
muaggatt_sub.head(2)

,mukey,muname,drclassdcd,forpehrtdcp
0,1723640,"Quillamook complex, 0 to 7 percent slopes",Well drained,Moderate
1,1723639,"Mosscreek-Fawceter complex, 30 to 60 percent s...",Well drained,Severe


In [52]:
muaggatt_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9749 entries, 0 to 9748
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mukey         9749 non-null   object 
 1   muname        9749 non-null   object 
 2   drclassdcd    9515 non-null   object 
 3   forpehrtdcp   9749 non-null   object 
 4   slopegraddcp  9664 non-null   float64
dtypes: float64(1), object(4)
memory usage: 380.9+ KB


The component table will just be used in relation to other tables reported at the component level (a subset of map unit).

In [18]:
# all we need from this table is the mukey - cokey crosswalk
# and the component percentage of the map unit
component_sub = component[['mukey', 'cokey', 'comppct_r']]
component_sub['comppct_p'] = component_sub['comppct_r']/100
component_sub.head(3)

C:\Users\sloreno\AppData\Local\Temp\ipykernel_33868\2825768937.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  component_sub['comppct_p'] = component_sub['comppct_r']/100


,mukey,cokey,comppct_r,comppct_p
0,1723640,22389538,25,0.25
1,1723640,22389539,60,0.60
2,1723640,22389540,2,0.02


The Component Forest Productivity table contains the site index information.

In [23]:
#first remove all null values from table
coforprod_drop = coforprod[coforprod['siteindex_r'].notna()]
coforprod_drop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 7827 entries, 4 to 23621
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   plantsym       7827 non-null   object  
 1   plantsciname   7827 non-null   object  
 2   plantcomname   7827 non-null   object  
 3   siteindexbase  7675 non-null   object  
 4   siteindex_l    2 non-null      float64 
 5   siteindex_r    7827 non-null   float64 
 6   siteindex_h    2 non-null      float64 
 7   fprod_l        2 non-null      float64 
 8   fprod_r        7086 non-null   float64 
 9   fprod_h        3 non-null      float64 
 10  cokey          7827 non-null   object  
 11  cofprodkey     7827 non-null   object  
 12  geometry       0 non-null      geometry
dtypes: float64(6), geometry(1), object(6)
memory usage: 856.1+ KB


In [87]:
# join with component table to see if there are multiple values per map unit
coforprod_key = coforprod_drop.merge(component_sub, on='cokey', how='left')
coforprod_key = coforprod_key.astype({'siteindex_r':'int'})
# if multiple values, grab first one
si_key = coforprod_key.groupby('mukey').first().reset_index()

In [88]:
si_key['si_label'] = (si_key.apply(lambda x: "{} - {} ft".format(x.plantcomname, x.siteindex_r), axis=1))
si_key = si_key[['mukey', 'si_label']]

The Component Restrictions table for depth to restrictive layer information

In [70]:
# in this table we need the cokey, 
# resdept_l - min depth to restrictive layer
# resdept_h - max depth to restrictive layer
corest_sub = corestrictions[['cokey','resdept_l', 'resdept_h', 'resdept_r']]
corest_sub.head(2)

,cokey,resdept_l,resdept_h,resdept_r
0,22389521,51.0,102.0,94
1,22389522,30.0,51.0,48


In [51]:
# merge the corest_sub and component
res = component_sub.merge(corest_sub, on='cokey', how='left')
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28555 entries, 0 to 28554
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   mukey      28555 non-null  object 
 1   cokey      28555 non-null  object 
 2   comppct_r  28555 non-null  int64  
 3   comppct_p  28555 non-null  float64
 4   resdept_l  17148 non-null  float64
 5   resdept_h  17123 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.3+ MB


In [86]:
# let's calcuate the percentage weighted values and sum by mukey
res['l_per'] = res['resdept_l']*res['comppct_p']
res['h_per'] = res['resdept_h']*res['comppct_p']

res.head(10)

,mukey,cokey,comppct_r,comppct_p,resdept_l,resdept_h,l_per,h_per
0,1723640,22389538,25,0.25,NaN,NaN,NaN,NaN
1,1723640,22389539,60,0.60,102.0,152.0,61.0,91.0
2,1723640,22389540,2,0.02,NaN,NaN,NaN,NaN
3,60935,22389548,45,0.45,51.0,102.0,23.0,46.0
4,60935,22389549,35,0.35,102.0,152.0,36.0,53.0
5,60934,22389566,45,0.45,51.0,102.0,23.0,46.0
6,60934,22389567,35,0.35,102.0,152.0,36.0,53.0
7,60933,22389693,8,0.08,NaN,NaN,NaN,NaN
8,60933,22389694,50,0.50,NaN,NaN,NaN,NaN
9,60933,22389695,35,0.35,NaN,NaN,NaN,NaN


In [69]:
res_comp = res.groupby('mukey').agg({'l_per':sum, 'h_per':sum}).reset_index()
res_comp.loc[res_comp['mukey']=='1723640']

,mukey,l_per,h_per
729,1723640,61.2,91.2


## Let's join our tables together

In [89]:
join = muaggatt_sub.merge(si_key, on='mukey', how='left')
join = join.merge(res_comp, on='mukey', how='left')
join['drclassdcd'].fillna("No Data Available", inplace=True)
join['si_label'].fillna('None', inplace=True)
join.rename(columns={'l_per': 'avg_rs_l', 'h_per': 'avg_rs_h'}, inplace=True)
join.head(10)

,mukey,muname,drclassdcd,forpehrtdcp,si_label,avg_rs_l,avg_rs_h
0,1723640,"Quillamook complex, 0 to 7 percent slopes",Well drained,Moderate,None,61.20,91.2
1,1723639,"Mosscreek-Fawceter complex, 30 to 60 percent s...",Well drained,Severe,western hemlock - 103 ft,45.90,68.4
2,1723638,"Killam-Fawceter-Rock outcrop complex, 60 to 90...",Well drained,Severe,western hemlock - 100 ft,48.45,81.3
3,60939,Water,No Data Available,Not rated,None,0.00,0.0
4,60938,"Caterl-Laderly complex, 30 to 60 percent slopes",Well drained,Severe,Douglas-fir - 112 ft,58.65,96.5
5,60937,"Caterl-Laderly complex, 3 to 30 percent slopes",Well drained,Severe,Douglas-fir - 113 ft,63.75,104.1
6,60936,"Brenner silt loam, 0 to 3 percent slopes",Poorly drained,Slight,None,0.00,0.0
7,60935,"Braun-Scaponia silt loams, 60 to 90 percent sl...",Well drained,Severe,Douglas-fir - 125 ft,58.65,99.1
8,60934,"Braun-Scaponia silt loams, 3 to 30 percent slopes",Well drained,Severe,Douglas-fir - 124 ft,58.65,99.1
9,60933,"Wauna-Locoda silt loams, protected, 0 to 3 per...",Poorly drained,Slight,None,0.00,0.0


## Bring in the map units shapefile and join with attributes

In [78]:
oregon = gpd.read_file(SOILS, driver="FileGDB", layer= 'MUPOlYGON')
oregon = oregon.to_crs(3857)
oregon.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 387800 entries, 0 to 387799
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   AREASYMBOL    387800 non-null  object  
 1   SPATIALVER    387800 non-null  float64 
 2   MUSYM         387800 non-null  object  
 3   MUKEY         387800 non-null  object  
 4   Shape_Length  387800 non-null  float64 
 5   Shape_Area    387800 non-null  float64 
 6   geometry      387800 non-null  geometry
dtypes: float64(3), geometry(1), object(3)
memory usage: 20.7+ MB


In [90]:
export = oregon.merge(join, left_on="MUKEY", right_on='mukey', how='left')
export.drop('mukey', axis=1, inplace = True)
export.insert(0, 'id', range(0, 0 + len(export)))
export.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 387800 entries, 0 to 387799
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            387800 non-null  int64   
 1   AREASYMBOL    387800 non-null  object  
 2   SPATIALVER    387800 non-null  float64 
 3   MUSYM         387800 non-null  object  
 4   MUKEY         387800 non-null  object  
 5   Shape_Length  387800 non-null  float64 
 6   Shape_Area    387800 non-null  float64 
 7   geometry      387800 non-null  geometry
 8   muname        387800 non-null  object  
 9   drclassdcd    387800 non-null  object  
 10  forpehrtdcp   387800 non-null  object  
 11  si_label      387800 non-null  object  
 12  avg_rs_l      387800 non-null  float64 
 13  avg_rs_h      387800 non-null  float64 
dtypes: float64(5), geometry(1), int64(1), object(7)
memory usage: 41.4+ MB


In [91]:
export.to_file('C:/Users/sloreno/LandMapper/data/Soil_Attributes/OR_soils.shp', driver='ESRI Shapefile')

C:\Users\sloreno\AppData\Local\Temp\ipykernel_33868\556916863.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  export.to_file('C:/Users/sloreno/LandMapper/data/Soil_Attributes/OR_soils.shp', driver='ESRI Shapefile')
